In [17]:
import pandas as pd
import numpy as np
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm
from scipy.stats import f_oneway
from scipy import stats

In [2]:
def is_non_zero_file(fpath):  
    if os.path.isfile(fpath):
        tmp = pd.read_csv(fpath)
        if len(tmp.columns)>1:
            return True
    return False

In [3]:
csv_dir = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/reports/New-Minerva-Data-output/yp2mf3i8_epoch=108-step=872.ckpt"

In [4]:
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

In [5]:
csv_files_df = [pd.read_csv(csv_file) for csv_file in csv_files if is_non_zero_file(csv_file)]

In [6]:
all_plaque_objects  = pd.concat(csv_files_df, axis=0, ignore_index=True)

In [7]:
all_plaque_objects["XENum"]=all_plaque_objects["image_name"].apply(lambda l: l.split("_1_")[0])

In [8]:
all_plaque_objects.head(2)

,image_name,label,confidence,brown_pixels,core,coarse_grained,diffuse,caa,centroid,eccentricity,area,equivalent_diameter,mask_present,total_processing_time,processing_time,XENum
0,XE10-032_1_AmyB_1x_7168_y_111616.png,CAA,0.818313,1332,0,0,0,1,"(81.05560418061062, 84.85475982083098)",0.785600,19423,157.258169,1,927.892682,NaN,XE10-032
1,XE10-032_1_AmyB_1x_21504_y_122880.png,Diffuse,0.989099,81,0,0,1,0,"(180.78206284247764, 190.18450440452244)",0.392877,115336,383.210590,1,927.892682,NaN,XE10-032


In [14]:
tile_count = all_plaque_objects.groupby(["XENum"])["image_name"].count().reset_index()
tile_count.columns = ["XENum","tile_count"]

In [16]:
tile_count

,XENum,tile_count
0,XE05-122,28
1,XE07-002,8844
2,XE07-027,190
3,XE07-033,16419
4,XE07-034,4815
...,...,...
293,XE19-049,6661
294,XE19-050,10
295,XE19-051,33
296,XE19-052,23


In [18]:
plaque_counts = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/csv_merged_data/Full_minerva_output.csv")
# create a patient id column to map with clinical metadata
plaque_counts["XENum"]=plaque_counts["image_name"].apply(lambda l: l.split("_1_")[0])

In [19]:
clinical_metadata = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/Clinical_correlation_analysis/Mapped_clinical_data_full_minerava.csv")

In [20]:
plaque_counts_merged = pd.merge(plaque_counts, clinical_metadata, on="XENum", how="left")

In [21]:
plaque_counts_merged["total_neuritic_plaques"] = plaque_counts_merged["total_core_plaques"]+plaque_counts_merged["total_coarse_grained_plaques"]

In [24]:
plaque_counts_merged = pd.merge(plaque_counts_merged,tile_count, how="left",on="XENum")

In [26]:
plaque_counts_merged["core_density"] = plaque_counts_merged["total_core_plaques"]/plaque_counts_merged["tile_count"]
plaque_counts_merged["diffuse_density"] = plaque_counts_merged["total_diffuse_plaques"]/plaque_counts_merged["tile_count"]
plaque_counts_merged["cg_density"] = plaque_counts_merged["total_coarse_grained_plaques"]/plaque_counts_merged["tile_count"]
plaque_counts_merged["caa_density"] = plaque_counts_merged["total_caa_plaques"]/plaque_counts_merged["tile_count"]

In [30]:
plaque_counts_merged["neuritic_density"] = (plaque_counts_merged["total_caa_plaques"]+plaque_counts_merged["total_coarse_grained_plaques"])/plaque_counts_merged["tile_count"]

In [31]:
plaque_counts_merged.head(2)

,image_name,total_core_plaques,total_coarse_grained_plaques,total_diffuse_plaques,total_caa_plaques,avg_area,avg_diameter,avg_eccentricity,age,sex,...,PMI,MCI,Dementia,total_neuritic_plaques,tile_count,core_density,diffuse_density,cg_density,caa_density,neuritic_density
0,XE10-032_1_AmyB_1,0,4,20,9,26962.61290,163.323817,0.604518,86,Female,...,6.083333,1.0,0.0,4,31,0.000000,0.645161,0.129032,0.290323,0.419355
1,XE14-033_1_AmyB_1,24,67,501,5,11890.09315,111.966076,0.556623,96,Male,...,11.416667,0.0,1.0,91,365,0.065753,1.372603,0.183562,0.013699,0.197260


In [36]:
df = plaque_counts_merged[(plaque_counts_merged["total_core_plaques"]<=2500) & (~plaque_counts_merged["MidPlaquesWCoresValue"].isna()) & (~plaque_counts_merged["Plq_Mn_MFG"].isna())]
df["MidPlaquesWCoresValue"] = df["MidPlaquesWCoresValue"].astype(int)
df["MidPlaquesWCoresValue_desc"] =  np.where(df["MidPlaquesWCoresValue"]==1,"Sparse",np.where(df["MidPlaquesWCoresValue"]==3,"Moderate", np.where(df["MidPlaquesWCoresValue"]==5,"Frequent","None")))
df["CERAD Ratings"] = df["MidPlaquesWCoresValue_desc"]
df  = df.sort_values(by="CERAD Ratings")
val_y = "Plq_Mn_MFG"
val_x= "core_density"
title = "Model Predicted - Total Neuritic plaques with Cores vs Average density of Neuritic plaques"
#fig = px.scatter(df, x=val_x, y=val_y, color="CERAD Ratings", trendline="ols", title=title, category_orders={"CERAD Ratings": ["Sparse","Moderate","Frequent"]},trendline_scope="overall",trendline_color_override="black")
fig = px.scatter(df, x=val_x, y=val_y,  trendline="ols", title=title,trendline_scope="overall",trendline_color_override="black")

fig.update_layout(height = 700, width = 900)
fig.update_layout( plot_bgcolor='white')
fig.update_xaxes( mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
"""
fig.add_annotation(x=1000, y=20,
            text="R-squared : 0.6",
            showarrow=True,
            arrowhead=1,
            font=dict(
                color="green",
                size=12
            ),arrowcolor="green")
"""
fig.update_layout(
    xaxis_title_text='Model Predicted - Total Neuritic plaques with Cores', # xaxis label
    yaxis_title_text="Average density of Neuritic plaques") # yaxis label
fig.show()


#results = px.get_trendline_results(fig)
#print(results.sumaary())

output = sm.OLS(df[val_y], df[val_x]).fit() 
print(output.summary())

/tmp/ipykernel_111325/2904423435.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_111325/2904423435.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_111325/2904423435.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



                                 OLS Regression Results                                
Dep. Variable:             Plq_Mn_MFG   R-squared (uncentered):                   0.432
Model:                            OLS   Adj. R-squared (uncentered):              0.429
Method:                 Least Squares   F-statistic:                              169.3
Date:                Fri, 06 Sep 2024   Prob (F-statistic):                    3.60e-29
Time:                        14:29:48   Log-Likelihood:                         -821.97
No. Observations:                 224   AIC:                                      1646.
Df Residuals:                     223   BIC:                                      1649.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------